<a href="https://colab.research.google.com/github/mathu3004/Pearl_Path/blob/Chatbot/ChatbotModelTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
test_dataset = [
    # Emergency Assistance
    ("What is the nearest hospital in Kandy?", "emergency_assistance"),
    ("How do I perform CPR?", "emergency_assistance"),
    ("Contact number for the police station in Colombo?", "emergency_assistance"),

    # Information Hub
    ("Tell me about the history of the Temple of the Tooth.", "information_hub"),
    ("When was the Nuwara Eliya Botanical Garden built?", "information_hub"),
    ("What are the opening hours of the National Museum of Colombo?", "information_hub"),

    # User-Provider Connection
    ("What is the phone number of Hotel X in Kandy?", "user_provider_connection"),
    ("Provide the email of Restaurant Y in Ella.", "user_provider_connection"),
    ("Amenities of Hotel Z in Nuwara Eliya.", "user_provider_connection"),

    # Recommendations
    ("Recommend me 7 hotels in Kandy.", "recommendations"),
    ("Suggest some restaurants in Ella.", "recommendations"),
    ("What are the best attractions in Colombo?", "recommendations"),

    # Chatbot Information
    ("What is your name?", "chatbot_info"),
    ("Tell me about the trip itinerary application.", "chatbot_info"),
    ("How can you assist me with my trip planning?", "chatbot_info"),

    # Unknown
    ("What is the weather like today?", "unknown"),
    ("Can you tell me a joke?", "unknown"),
    ("What are the latest news headlines?", "unknown")
]
